<a href="https://colab.research.google.com/github/lainey-reed/Textbook/blob/master/Colab/AC_Textbook_Chapter11.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Chapter 11: Hydraulics

This Colab document corresponds to the eleventh chapter of the AguaClara textbook.

In [ ]:
!pip install aguaclara

#Constants Needed To Run Snippets:

In [ ]:
import aguaclara.core.physchem as pc
import aguaclara.core.utility as uc
import aguaclara.core.pipes as pp


from aguaclara.core.units import unit_registry as u

import numpy as np
import matplotlib.pyplot as plt
#I played with the velocity gradient to get a reasonable head loss of 50 cm.
G_CS = 123 * u.Hz
density_clay = 2650 * u.kg/u.m**3
#From William's research the k for flocculator is 0.03. We need to confirm that this value is correct.
k = 0.03
# It isn't clear what value should be used for alpha
alpha = 0.5
# The following 3 values are estimates from Casey's floc filter video.
C_infb = 40 * u.NTU
C_outfb = 1 * u.NTU
t_fb = 800 * u.s
k_fb = -1/t_fb*(np.log(C_outfb/C_infb))
C_out_f = (np.pi* density_clay/6 * (k_fb/(np.pi*alpha*k*G_CS))**(3/2)).to(u.NTU)

Gt = (3/2 * 1/(k*np.pi * alpha) * (np.pi/6 * density_clay/C_out_f)**(2/3)).to(u.dimensionless)
t_floc = 3/2 * 1/k_fb

Temp = 20 * u.degC

Floc_HL = (G_CS**2 * t_floc * pc.viscosity_kinematic(Temp)/u.standard_gravity).to(u.cm)

#Example: Inlet Manifold Flow Distribution 

In [ ]:
def Ratio_Pipe_Manifold_V_Port_to_V_Man(Ratio_Flow):
  return np.sqrt((Ratio_Flow**2 + 1)/(2*(1-Ratio_Flow**2)))

#Example: Sedimentation Tank Inlet Manifold

In [ ]:
#Design the sedimentation tank inlet manifold
v_FB = 1 * u.mm/u.s
W_Diffuser = 3.175 * u.mm
W_Sed = 1.07 * u.m
v_jet = v_FB * W_Sed/W_Diffuser
print('The jet velocity is',v_jet)
L_Sed = 6 * u.m
Ratio_Flow = 0.85
v_Influent_Manifold = v_jet / Ratio_Pipe_Manifold_V_Port_to_V_Man(Ratio_Flow)
print('The manifold velocity is',v_Influent_Manifold)
Q_sed =(L_Sed * W_Sed * v_FB).to(u.L/u.s)
ID_min_Influent_Manifold = pc.diam_circle(Q_sed/v_Influent_Manifold)
print('The minimum manifold diameter is',ID_min_Influent_Manifold.to(u.inch))
SDR = 41
ID_Influent_Manifold = uc.ceil_nearest(ID_min_Influent_Manifold,pp.ID_SDR_all_available(SDR))
ND_Influent_Manifold = pp.ND_SDR_available(ID_Influent_Manifold,SDR)
print('The manifold nominal diameter is',ND_Influent_Manifold.to(u.inch))

#Example: Sedimentation Tank Outlet Manifold

In [ ]:
#Design the sedimentation tank inlet manifold
HL_orifice = 5 * u.cm
v_orifice_contracted = np.sqrt(2 * u.gravity * HL_orifice)
v_Effluent_Manifold = (v_orifice_contracted / Ratio_Pipe_Manifold_V_Port_to_V_Man(Ratio_Flow)).to(u.m/u.s)
print('The maximum effluent manifold velocity is',v_Effluent_Manifold)
Q_sed =(L_Sed * W_Sed * v_FB).to(u.L/u.s)
ID_min_Effluent_Manifold = pc.diam_circle(Q_sed/v_Effluent_Manifold)
print('The minimum effluent manifold diameter is',ID_min_Effluent_Manifold.to(u.inch))
SDR = 41
ID_Effluent_Manifold = uc.ceil_nearest(ID_min_Effluent_Manifold,pp.ID_SDR_all_available(SDR))
ND_Effluent_Manifold = pp.ND_SDR_available(ID_Effluent_Manifold,SDR)
print('The manifold nominal diameter is',ND_Effluent_Manifold.to(u.inch))

#Example: Sedimentor Inlet Channel

In [ ]:
Pi_Q_min = 0.8
Pi_Q = np.linspace(Pi_Q_min,0.99,50)
Psi_Sed = 5 * u.cm

def v_man(Psi,Pi_Q):
  v_man = (2 * np.sqrt(u.gravity * Psi * (1 - Pi_Q**2)/(Pi_Q**2 + 1) )).to(u.m/u.s)
  return v_man

v_man(Psi_Sed,Pi_Q)

plt.plot(Pi_Q,v_man(Psi_Sed,Pi_Q),linewidth=2, color='blue')
plt.plot([Pi_Q_min,1],[0.45,0.45],linewidth=2, color='black')
plt.plot([Pi_Q_min,1],[0.15,0.15],linewidth=2, color='black', linestyle='dashed')
plt.ylabel('Channel water velocity (m/s)')
plt.xlabel('Flow uniformity')
plt.ylim((0,0.7))
plt.legend(['Max channel velocity','10 State Standards Max','10 State Standards Min'])
plt.show()

#Example: Filter Inlet Channel with Rectangular Weir Flow Distribution

In [ ]:
def Inlet_Channel_V_Max(H_weir,Ratio_Flow):
  return (2 * np.sqrt(u.gravity*H_weir*(1-Ratio_Flow**(2/3))/(1+Ratio_Flow**(2/3)))).to(u.m/u.s)

Ratio_Q_graph = np.linspace(0.6,0.95,20)

H_weir = 5 * u.cm
v_graph = np.empty_like(Ratio_Q_graph) * u.m/u.s
for i in range(20):
  v_graph[i] = Inlet_Channel_V_Max(H_weir,Ratio_Q_graph[i])

plt.plot(Ratio_Q_graph,v_graph)
plt.xlabel(r'Flow ratio, $\Pi_{Q_{weir}}$')
plt.ylabel(r'Maximum manifold channel velocity, $ \bar v_{M_1} \left(\frac{m}{s} \right)$')
plt.show()